In [1]:
# Install Libraries
!pip install langdetect
!pip install pycountry
!pip install textblob
!pip install tweepy
!pip install wordcloud

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
# Import Libraries
from textblob import TextBlob
import sys
import tweepy
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import nltk
import pycountry
import re
import string
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from langdetect import detect
from nltk.stem import SnowballStemmer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\zjw64\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [3]:
# Authentication
apiKey = "mi3IsHw1lzA9KlstGoEi4sQSu"
apiSecret = "0obn2WECJqrZt5eoYJoOY5HbHCQfa5Xe6PFW58EG3AEqF4dxUM"
accessToken = "1441571989357957122-eksbtpbuyFyQiMnIm9t3DOkILvzXLV" 
accessTokenSecret = "aLssavUn0tPln5czeoWaVcf0UYSRYAoenbyBIdf7TPvBE"
auth = tweepy.OAuthHandler(apiKey, apiSecret)
auth.set_access_token(accessToken, accessTokenSecret)
api = tweepy.API(auth)

In [4]:
# search tweets
keyword = input("Please enter keyword to search: ")
numOfTweet = int(input ("Please enter the number of tweets to analyze: "))

tweets = tweepy.Cursor(api.search_tweets, q=keyword).items(numOfTweet)
tweet_list = []
for tweet in tweets:
    tweet_list.append(tweet.text)

Please enter keyword to search: red sox
Please enter the number of tweets to analyze: 1500


In [5]:
tweet_list = pd.DataFrame(tweet_list)
print("total number: ",len(tweet_list))

total number:  1500


In [6]:
# delete duplicates
tweet_list.drop_duplicates(inplace = True)
print("total number: ",len(tweet_list))

total number:  946


In [7]:
#Extracting text values
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]
tw_list

,0,text
0,RT @TalkinBaseball_: Cardinals vs. Dodgers and...,RT @TalkinBaseball_: Cardinals vs. Dodgers and...
1,@marcgilman @Trifected @draysbay Ok. Whatever ...,@marcgilman @Trifected @draysbay Ok. Whatever ...
2,RT @tomperreira: Looking more and more like Re...,RT @tomperreira: Looking more and more like Re...
3,@nyyankeechick I'm watching the Yankees sweep ...,@nyyankeechick I'm watching the Yankees sweep ...
4,RT @tomperreira: The Red Sox will win the next...,RT @tomperreira: The Red Sox will win the next...
...,...,...
1491,"Stanton's slam lifts Yanks to 5-3 win, WC tie ...","Stanton's slam lifts Yanks to 5-3 win, WC tie ..."
1493,Last time Oregon State won at USC Eisenhower w...,Last time Oregon State won at USC Eisenhower w...
1495,Yankees have flipped the script on Red Sox htt...,Yankees have flipped the script on Red Sox htt...
1497,"Stanton's slam lifts Yanks to 5-3 win, WC tie ...","Stanton's slam lifts Yanks to 5-3 win, WC tie ..."


In [8]:
#Cleaning Text (RT, Punctuation etc)

#Creating new dataframe and new features
tw_list = pd.DataFrame(tweet_list)
tw_list["text"] = tw_list[0]

#Removing RT, Punctuation etc
remove_rt = lambda x: re.sub('RT @\w+: '," ",x)
rt = lambda x: re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",x)
tw_list["text"] = tw_list.text.map(remove_rt).map(rt)
tw_list["text"] = tw_list.text.str.lower()
tw_list.head(10)

,0,text
0,RT @TalkinBaseball_: Cardinals vs. Dodgers and...,cardinals vs dodgers and yankees vs red sox...
1,@marcgilman @Trifected @draysbay Ok. Whatever ...,ok whatever you want to say i ve seen ...
2,RT @tomperreira: Looking more and more like Re...,looking more and more like red sox and blue j...
3,@nyyankeechick I'm watching the Yankees sweep ...,i m watching the yankees sweep the red sox i...
4,RT @tomperreira: The Red Sox will win the next...,the red sox will win the next two to take the...
5,RT @BarstoolHubbs: Red Sox broadcast of the Gi...,red sox broadcast of the giancarlo grand slam...
6,RT @BryanOreally: Beautiful day for the Jets t...,beautiful day for the jets to pull the upset ...
7,"Pats on the big screens today, Red Sox tonight...",pats on the big screens today red sox tonight...
8,RT @hoodiegarysanc1: Red Sox broadcast of the ...,red sox broadcast of the giancarlo grand slam...
9,RT @nyyankeesstats: Yankees 2021 Grand Slams\n...,yankees 2021 grand slams giancarlo stanton 4...


In [9]:
#Calculating Negative, Positive, Neutral and Compound values

tw_list[['polarity', 'subjectivity']] = tw_list['text'].apply(lambda Text: pd.Series(TextBlob(Text).sentiment))
for index, row in tw_list['text'].iteritems():
    score = SentimentIntensityAnalyzer().polarity_scores(row)
    neg = score['neg']
    neu = score['neu']
    pos = score['pos']
    comp = score['compound']
    if neg > pos:
        tw_list.loc[index, 'sentiment'] = "negative"
    elif pos > neg:
        tw_list.loc[index, 'sentiment'] = "positive"
    else:
        tw_list.loc[index, 'sentiment'] = "neutral"
    tw_list.loc[index, 'neg'] = neg
    tw_list.loc[index, 'neu'] = neu
    tw_list.loc[index, 'pos'] = pos
    tw_list.loc[index, 'compound'] = comp

tw_list.head(10)

,0,text,polarity,subjectivity,sentiment,neg,neu,pos,compound
0,RT @TalkinBaseball_: Cardinals vs. Dodgers and...,cardinals vs dodgers and yankees vs red sox...,-0.300000,0.466667,negative,0.144,0.856,0.000,-0.4019
1,@marcgilman @Trifected @draysbay Ok. Whatever ...,ok whatever you want to say i ve seen ...,0.500000,0.500000,positive,0.000,0.800,0.200,0.3612
2,RT @tomperreira: Looking more and more like Re...,looking more and more like red sox and blue j...,0.250000,0.275000,positive,0.000,0.821,0.179,0.4690
3,@nyyankeechick I'm watching the Yankees sweep ...,i m watching the yankees sweep the red sox i...,0.400000,0.350000,negative,0.215,0.613,0.172,-0.1779
4,RT @tomperreira: The Red Sox will win the next...,the red sox will win the next two to take the...,0.266667,0.133333,positive,0.000,0.798,0.202,0.5859
5,RT @BarstoolHubbs: Red Sox broadcast of the Gi...,red sox broadcast of the giancarlo grand slam...,0.250000,0.500000,positive,0.191,0.588,0.221,0.1027
6,RT @BryanOreally: Beautiful day for the Jets t...,beautiful day for the jets to pull the upset ...,0.425000,0.500000,positive,0.116,0.711,0.173,0.3182
7,"Pats on the big screens today, Red Sox tonight...",pats on the big screens today red sox tonight...,0.000000,0.050000,neutral,0.000,1.000,0.000,0.0000
8,RT @hoodiegarysanc1: Red Sox broadcast of the ...,red sox broadcast of the giancarlo grand slam...,0.250000,0.500000,positive,0.191,0.588,0.221,0.1027
9,RT @nyyankeesstats: Yankees 2021 Grand Slams\n...,yankees 2021 grand slams giancarlo stanton 4...,0.500000,1.000000,positive,0.000,0.850,0.150,0.4588


In [10]:
#Creating new data frames for all sentiments (positive, negative and neutral)

tw_list_negative = tw_list[tw_list["sentiment"]=="negative"]
tw_list_positive = tw_list[tw_list["sentiment"]=="positive"]
tw_list_neutral = tw_list[tw_list["sentiment"]=="neutral"]

In [11]:
#Function for count_values_in single columns

def count_values_in_column(data,feature):
    total=data.loc[:,feature].value_counts(dropna=False)
    percentage=round(data.loc[:,feature].value_counts(dropna=False,normalize=True)*100,2)
    return pd.concat([total,percentage],axis=1,keys=['Total','Percentage'])

In [12]:
#Count_values for sentiment
count_values_in_column(tw_list,"sentiment")

,Total,Percentage
positive,436,46.09
neutral,333,35.20
negative,177,18.71
